# Modell

## Komponenten

```{figure} ./images/model_aufbau.png
:name: model_aufbau
:align: center
Komponenten des Segment Anything Models. Image Encoder, Prompt Encoder, and Mask Decoder. {cite}`kirillov2023segment`
```

### 1. Image Encoder
- Erstellt **Image Embeddings** für Input-Bilder
- In Theorie kann jede Art von Image Encoder verwendet werden
    - Vorausgesetzt Output ist ein (C×H×W) Embedding
- **Vision Transformer** (ViT) von {cite}`dosovitskiy2021image`. Angepasst für hochauflösende Input Bilder
    - pre-trained mit **Masked Autoencoder** (MAE) Verfahren
    - ViT-H/16
    - 14x14 windowed attention Blocks
    - 4 global attention Blocks

```{figure} ./images/ImageEncoderDiagram.png
:name: ImageEncoderDiagram
:align: center
SAM Image Decoder Struktur im Detail. {cite}`kirillov2023segment`. Eigene Darstellung
```

### 2. Prompt Encoder
- Prompts werden je nach Art unterschiedlich encoded

| Type   | Prompt | Embedding                                  |
|--------|--------|--------------------------------------------|
| Sparse | Points | Positional Encoding + gelernte Embeddings |
| Sparse | Boxes  | Positional Encoding + gelernte Embeddings |
| Sparse | Text   | CLIP Encoder                               |
| Dense  | Mask   | Convolution Embedding + Image Embedding    |

- Sparse Prompts werden auf ein **256-dimensional vectorial Embedding** gemappt:
    - **Points:** Positional Encodings der Koordinaten summiert mit trainierten Embeddings für Vorder- bzw. Hintergrund.
    - **Box:** Embedding Paar: Positional Encoding von Koordinaten "Oben Links" und "Unten Rechts" werden mit zugehörigen gelernten Embeddings summiert.
    - **Text:** CLIP Encoder. Jedoch jeder Text Encoder theoretisch möglich.
- **Masks** (Dense Prompts) werden gedownscaled und durch mehrere Convolution Layers transformiert (Siehe Abbildung).
 
```{figure} ./images/MaskPromptEncoding.png
:name: MaskPromptEncoder
:align: center
SAM Mask Prompt Encoding Struktur. {cite}`kirillov2023segment`. Eigene Darstellung
```


### 3. Mask Decoder
- Mappt Image Encoding, Prompt Encoding und ein Output Token auf eine **Maske**.
    - Vor Decoding wird dem Prompt Embedding ein trainiertes **Output Token Embedding** hinzugefügt.
    - Das **Output Token Embedding** ist eine lernbare Repräsentation der Output Maske.
- Modifizierter **Transformer Decoder Block** gefolgt von einem **Dynamic Mask Prediction Head**.
 
- Inspiriert von Transformer Architekturen von {cite}`carion2020endtoend` und {cite}`cheng2021perpixel`.

```{figure} ./images/mask_decoder_model.png
:name: mask_decoder_model
:align: center
SAM Lightweight Mask Decoder. {cite}`kirillov2023segment`
```

#### Decoder Aufbau
- **2 Decoder Layer** kombinieren Informationen aus Image- und Prompt Encodings.
    - Self-Attention wird auf Tokens (Prompt Encoding + Output Token) angewendet.
    - Cross-Attention zwischen Image Encoding und Tokens in beide Richtungen.
- Anschließendes 4x upscaling der Image Embeddings und weitere Cross-Attention ausgehend von den Token Embeddings.
- **Dynamic Mask Prediction Head** gibt Maske als Output aus.
- **Zweiter Head** wird trainiert den **IoU** (Intersection Over Union) des Outputs und der Ground-Truth Maske vorherzusagen.



`````{admonition} Cross-Attention in SAM
:class: tip
Im Mask Decoder von SAM wird **Cross-Attention in beide Richtungen** angewendet (Image to Prompt & Prompt to Image).
Am Beispiel Prompt to Image werden **Key** und **Value** anhand von Image Embeddings trainiert während **Query** von Prompt Embeddings selbst gelernt wird. 
`````


`````{admonition} Vorhersage von mehreren Masken
:class: tip
Um auf die **Mehrdeutigkeit von Prompts** eingehen zu können, können mit SAM mehrere Masken gleichzeitig für einen Prompt berechnet werden. Dazu werden, anstatt einem, **mehrere Output Tokens** an die Prompt Encodings zu Beginn des Decoders angehängt. Pro Output Token wird demnach eine Output Maske berechnet.
`````

#### Erkenntnisse während des Trainings
- Um ausreichend **geometrische Informationen** im Encoder zu erhalten, müssen Image Embeddings immer wenn diese in einem Attention Layer vorkommen mit **Positional Encodings** versehen werden.
- Die **originalen Prompt Tokens** (vor Eingabe im Decoder) werden bei jeder Aktualisierung der Tokens (Prompt Encoding + Output Token) im Decoder zusätzlich hinzugefügt um besser **Informationen über den Typ und die Position** des Prompts zu erhalten: *"This allows for a strong dependence on both the prompt token’s geometric location and type"*.{cite}`kirillov2023segment`.


## Training

- Nutzung eines **Interactive Segmentation Setup** angelehnt an {cite}`sofiiuk2021reviving` und {cite}`forte2020getting`.
- **11 Interationen** pro Trainingsschritt:
    - Erste Prediction mit Bounding Box oder Point Prompt.
    - 8 Iterative Predictions mit **vorheriger Output Maske** und **gesampelten Punkten** der Prediction Error Region.
    - Zwei zusätzliche Iterationen ohne zusätzliche Punkte (Nur vorherige Output Maske).
    
`````{admonition} Lightweight Mask Decoder
:class: note
Durch die gute Performance des Decoders konnten in SAM deutlich **mehr Iterationen pro Trainingsschritt** als in vorherigen Interactive Segmentation Projekten verwendet werden.
`````


### Trainingsablauf
#### Erste Iteration
```{figure} ./images/step1.png
:name: training_iterations_1
:align: center
Erste Maskenvorhersage eines Trainingsschritts. Eigene Darstellung
```

#### Vergleich mit der Ground Truth
```{figure} ./images/step2.png
:name: training_iterations_2
:align: center
Vergleich der ersten Maske mit der Ground Truth Maske. Eigene Darstellung
```

#### Zweite Iteration
```{figure} ./images/step3.png
:name: training_iterations_3
:align: center
Zweite Maskenvorhersage mit zusätzlichen Prompts aus dem ersten Output. Eigene Darstellung
```

- Um das Model auf jegliche Arten von Anwendungen zu traineren, wurden initiale Box Prompts mit **Random-Noise** versehen, um sowohl mit Boxen zu trainieren, die zu klein als auch zu groß für das Zielobjekt sind.
- Um mehr Informationen im Training zu erhalten, wurden **Masken Input Prompts als Wahrscheinlichkeiten** und nicht als binäre Werte in den Decoder übergeben.

### Trainingsparameter
- AdamW Optimizer
- Initiale Lernrate von 0.0008
- Batchsize von 256 Bildern
- Layer-wise Lernrate Decay von 0.8
- Weight Decay von 0.1

`````{admonition} Loss
:class: note
SAM benutzt eine Kombination von **Focal Loss** und **Dice Loss**, um seine Gewichte zu trainieren.
`````

&rarr; SAM wird in der finalen Version für **90.000 Iterationen** trainiert (ca. 2 SA-1B Epochen)